## NYT COVID-19 Analysis
This notebook processes and performs quick analysis from the [New York Times COVID-19 dataset](https://github.com/nytimes/covid-19-data).  The data is updated in the `/databricks-datasets/COVID/covid-19-data/` location regularly so you can access the data directly.

In [2]:
# Standard Libraries
import io

# External Libraries
import requests
import numpy as np
import pandas as pd
import altair as alt
from vega_datasets import data

# topographical
topo_usa = 'https://vega.github.io/vega-datasets/data/us-10m.json'
topo_wa = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
topo_king = 'https://raw.githubusercontent.com/johan/world.geo.json/master/countries/USA/WA/King.geo.json'

/databricks/python/lib/python3.7/site-packages/jinja2/runtime.py:318: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping

### Download Mapping County FIPS to lat, long_

In [4]:
%sh mkdir -p /dbfs/tmp/dennylee/COVID/map_fips/ && wget -O /dbfs/tmp/dennylee/COVID/map_fips/countyfips_lat_long.csv https://raw.githubusercontent.com/dennyglee/tech-talks/master/datasets/countyfips_lat_long.csv && ls -al /dbfs/tmp/dennylee/COVID/map_fips/

--2020-04-17 22:32:47-- https://raw.githubusercontent.com/dennyglee/tech-talks/master/datasets/countyfips_lat_long.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 129178 (126K) [text/plain]
Saving to: ‘/dbfs/tmp/dennylee/COVID/map_fips/countyfips_lat_long.csv’

 0K .......... .......... .......... .......... .......... 39% 3.06M 0s
 50K .......... .......... .......... .......... .......... 79% 7.03M 0s
 100K .......... .......... ...... 100% 20.9M=0.02s

2020-04-17 22:32:47 (5.10 MB/s) - ‘/dbfs/tmp/dennylee/COVID/map_fips/countyfips_lat_long.csv’ saved [129178/129178]

total 135
drwxrwxrwx 2 root root 4096 Apr 17 2020 .
drwxrwxrwx 2 root root 4096 Apr 17 22:29 ..
-rwxrwxrwx 1 root root 129178 Apr 17 2020 countyfips_lat_long.csv

In [5]:
# Create mapping of county FIPS to centroid long_ and lat
map_fips = spark.read.option("header", True).option("inferSchema", True).csv("/tmp/dennylee/COVID/map_fips/countyfips_lat_long.csv")
map_fips = (map_fips
              .withColumnRenamed("STATE", "state")
              .withColumnRenamed("COUNTYNAME", "county")
              .withColumnRenamed("LAT", "lat")
              .withColumnRenamed("LON", "long_"))
map_fips.createOrReplaceTempView("map_fips")

In [6]:
map_fips_dedup = spark.sql("""select fips, min(state) as state, min(county) as county, min(long_) as long_, min(lat) as lat from map_fips group by fips""")
map_fips_dedup.createOrReplaceTempView("map_fips_dedup")

### Download 2019 Population Estimates

In [8]:
%sh mkdir -p /dbfs/tmp/dennylee/COVID/population_estimates_by_county/ && wget -O /dbfs/tmp/dennylee/COVID/population_estimates_by_county/co-est2019-alldata.csv https://raw.githubusercontent.com/databricks/tech-talks/master/datasets/co-est2019-alldata.csv && ls -al /dbfs/tmp/dennylee/COVID/population_estimates_by_county/

--2020-04-17 22:33:51-- https://raw.githubusercontent.com/databricks/tech-talks/master/datasets/co-est2019-alldata.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3644730 (3.5M) [text/plain]
Saving to: ‘/dbfs/tmp/dennylee/COVID/population_estimates_by_county/co-est2019-alldata.csv’

 0K .......... .......... .......... .......... .......... 1% 3.71M 1s
 50K .......... .......... .......... .......... .......... 2% 7.98M 1s
 100K .......... .......... .......... .......... .......... 4% 7.40M 1s
 150K .......... .......... .......... .......... .......... 5% 23.2M 0s
 200K .......... .......... .......... .......... .......... 7% 28.3M 0s
 250K .......... .......... .......... .......... .......... 8% 11.5M 0s
 300K .......... .......... .......... .......... .......... 9% 30.3M 0s
 350K .......... .......... .......... .......... .......... 11% 25.4M 0s
 400K .......... .......... .......... .......... .......... 12% 26.5M 0s
 450K .......... .......... .......... .......... .......... 14% 26.5M 0s
 500K .......... .......... .......... .......... .......... 15% 31.8M 0s
 550K .......... .......... .......... .......... .......... 16% 25.2M 0s
 600K .......... .......... .......... .......... .......... 18% 24.4M 0s
 650K .......... .......... .......... .......... .......... 19% 29.7M 0s
 700K .......... .......... .......... .......... .......... 21% 30.2M 0s
 750K .......... .......... .......... .......... .......... 22% 24.2M 0s
 800K .......... .......... .......... .......... .......... 23% 28.2M 0s
 850K .......... .......... .......... .......... .......... 25% 31.2M 0s
 900K .......... .......... .......... .......... .......... 26% 30.7M 0s
 950K .......... .......... .......... .......... .......... 28% 18.0M 0s
 1000K .......... .......... .......... .......... .......... 29% 26.6M 0s
 1050K .......... .......... .......... .......... .......... 30% 30.8M 0s
 1100K .......... .......... .......... .......... .......... 32% 29.9M 0s
 1150K .......... .......... .......... .......... .......... 33% 25.9M 0s
 1200K .......... .......... .......... .......... .......... 35% 30.5M 0s
 1250K .......... .......... .......... .......... .......... 36% 31.8M 0s
 1300K .......... .......... .......... .......... .......... 37% 33.4M 0s
 1350K .......... .......... .......... .......... .......... 39% 26.7M 0s
 1400K .......... .......... .......... .......... .......... 40% 29.2M 0s
 1450K .......... .......... .......... .......... .......... 42% 31.7M 0s
 1500K .......... .......... .......... .......... .......... 43% 31.2M 0s
 1550K .......... .......... .......... .......... .......... 44% 26.4M 0s
 1600K .......... .......... .......... .......... .......... 46% 24.7M 0s
 1650K .......... .......... .......... .......... .......... 47% 30.9M 0s
 1700K .......... .......... .......... .......... .......... 49% 31.5M 0s
 1750K .......... .......... .......... .......... .......... 50% 31.5M 0s
 1800K .......... .......... .......... .......... .......... 51% 26.6M 0s
 1850K .......... .......... .......... .......... .......... 53% 32.0M 0s
 1900K .......... .......... .......... .......... .......... 54% 30.8M 0s
 1950K .......... .......... .......... .......... .......... 56% 31.8M 0s
 2000K .......... .......... .......... .......... .......... 57% 26.8M 0s
 2050K .......... .......... .......... .......... .......... 59% 31.7M 0s
 2100K .......... .......... .......... .......... .......... 60% 29.8M 0s
 2150K .......... .......... .......... .......... .......... 61% 31.4M 0s
 2200K .......... .......... .......... .......... .......... 63% 26.9M 0s
 2250K .......... .......... .......... .......... .......... 64% 31.0M 0s
 2300K .......... .......... .......... .......... .......... 66% 31.4M 0s
 2350K .......... ........

In [9]:
map_popest_county = spark.read.option("header", True).option("inferSchema", True).csv("/tmp/dennylee/COVID/population_estimates_by_county/co-est2019-alldata.csv")
map_popest_county.createOrReplaceTempView("map_popest_county")
fips_popest_county = spark.sql("select State * 1000 + substring(cast(1000 + County as string), 2, 3) as fips, STNAME, CTYNAME, census2010pop, POPESTIMATE2019 from map_popest_county")
fips_popest_county.createOrReplaceTempView("fips_popest_county")

## Specify `nyt_daily` table
* Source: `/databricks-datasets/COVID/covid-19-data/`
* Contains the COVID-19 daily reports

In [11]:
nyt_daily = spark.read.option("inferSchema", True).option("header", True).csv("/databricks-datasets/COVID/covid-19-data/us-counties.csv")
nyt_daily.createOrReplaceTempView("nyt_daily")
display(nyt_daily)

date,county,state,fips,cases,deaths
2020-01-21T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-22T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-23T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-24T00:00:00.000+0000,Cook,Illinois,17031,1,0
2020-01-24T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-25T00:00:00.000+0000,Orange,California,6059,1,0
2020-01-25T00:00:00.000+0000,Cook,Illinois,17031,1,0
2020-01-25T00:00:00.000+0000,Snohomish,Washington,53061,1,0
2020-01-26T00:00:00.000+0000,Maricopa,Arizona,4013,1,0
2020-01-26T00:00:00.000+0000,Los Angeles,California,6037,1,0


# COVID-19 Cases and Deaths for Specific Counties
Focusing on two-week window around when educational facilites were closed
* Top 10 Washington State counties (3/13/2020)
* Top 10 NY State counties (3/18/2020)
The queries are using the US Census Population Estimates for 2019

In [13]:
# WA State 2 week window
wa_state_window = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-06T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  from (
SELECT CAST(f.date AS date) AS date, f.county, f.fips, SUM(f.cases) AS cases, SUM(f.deaths) AS deaths, MAX(p.POPESTIMATE2019) AS population_estimate 
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = f.fips
 WHERE f.state = 'Washington' 
   AND date BETWEEN '2020-03-06T00:00:00.000+0000' AND '2020-03-20T00:00:00.000+0000'
 GROUP BY f.date, f.county, f.fips
) a""")
wa_state_window.createOrReplaceTempView("wa_state_window")

# NY State 2 week window
ny_state_window = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-11T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  FROM (
SELECT CAST(f.date AS date) AS date, f.county, p.fips, SUM(f.cases) as cases, SUM(f.deaths) as deaths, MAX(p.POPESTIMATE2019) AS population_estimate  
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = coalesce(f.fips, 36061)
 WHERE f.state = 'New York' 
   AND date BETWEEN '2020-03-11T00:00:00.000+0000' AND '2020-03-25T00:00:00.000+0000'
 GROUP BY f.date, f.county, p.fips
) a""")
ny_state_window.createOrReplaceTempView("ny_state_window")

# NY State 2 week window (-1 week)
ny_state_window_m1 = spark.sql("""
SELECT date, 100 + datediff(date, '2020-03-06T00:00:00.000+0000') as day_num, county, fips, cases, deaths, 100000.*cases/population_estimate AS cases_per_100Kpop, 100000.*deaths/population_estimate AS deaths_per_100Kpop
  FROM (
SELECT CAST(f.date AS date) AS date, f.county, p.fips, SUM(f.cases) as cases, SUM(f.deaths) as deaths, MAX(p.POPESTIMATE2019) AS population_estimate  
  FROM nyt_daily f 
    JOIN fips_popest_county p
      ON p.fips = coalesce(f.fips, 36061)
 WHERE f.state = 'New York' 
   AND date BETWEEN '2020-03-06T00:00:00.000+0000' AND '2020-03-20T00:00:00.000+0000'
 GROUP BY f.date, f.county, p.fips
) a""")
ny_state_window_m1.createOrReplaceTempView("ny_state_window_m1")

## COVID-19 Cases for WA and NY Counties

In [15]:
%sql
SELECT f.date, f.county, f.cases 
  FROM wa_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM wa_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-11,Island,1
2020-03-11,King,235
2020-03-12,Pierce,17
2020-03-13,Snohomish,132
2020-03-20,Island,19
2020-03-07,Snohomish,26
2020-03-15,Kitsap,5
2020-03-12,Spokane,4
2020-03-12,Snohomish,107
2020-03-15,Whatcom,3


In [16]:
%sql
SELECT f.date, f.county, f.cases 
  FROM ny_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-18,Nassau,183
2020-03-16,Suffolk,63
2020-03-22,Dutchess,82
2020-03-23,Orange,389
2020-03-21,Monroe,42
2020-03-24,Albany,146
2020-03-15,Dutchess,7
2020-03-25,Albany,152
2020-03-21,Albany,88
2020-03-15,Orange,6


In [17]:
%sql
SELECT f.date, f.county, f.cases 
  FROM ny_state_window_m1 f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window_m1 GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases
2020-03-18,Nassau,183
2020-03-16,Suffolk,63
2020-03-11,Saratoga,2
2020-03-15,Dutchess,7
2020-03-15,Orange,6
2020-03-19,New York City,2468
2020-03-07,Saratoga,2
2020-03-17,Suffolk,84
2020-03-18,Westchester,537
2020-03-08,Rockland,2


## COVID-19 Cases per 100K people for WA and NY Counties
The concern with the above graphs is while it provides you the total number of cases, it's hard to compare WA state and NY state due to density.  While not perfect, a better way to look at these numbers would be to review this data as a proportion of population estimates.

Let's look at these values by a percentage of the population; the numbers used are the 2019 US Census estimates of county populations.

*Note, reviewing the top 10 counties by case (vs. % of cases)*

In [19]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM wa_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM wa_state_window GROUP BY county ORDER BY cases DESC LIMIT 10  
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-11,Island,1.17452226307
2020-03-11,King,10.43154641683
2020-03-12,Pierce,1.87849455237
2020-03-13,Snohomish,16.05677285627
2020-03-20,Island,22.31592299832
2020-03-07,Snohomish,3.16269768381
2020-03-15,Kitsap,1.84180378896
2020-03-12,Spokane,0.76511386807
2020-03-12,Snohomish,13.01571739107
2020-03-15,Whatcom,1.30863217403


In [20]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM ny_state_window f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-18,Nassau,13.48638538341
2020-03-16,Suffolk,4.26655542018
2020-03-22,Dutchess,27.87049058861
2020-03-23,Orange,101.05470982491
2020-03-21,Monroe,5.66213246694
2020-03-24,Albany,47.78956878097
2020-03-15,Dutchess,2.37918822098
2020-03-25,Albany,49.75352366238
2020-03-21,Albany,28.80467159401
2020-03-15,Orange,1.55868447031


In [21]:
%sql
SELECT f.date, f.county, f.cases_per_100Kpop 
  FROM ny_state_window_m1 f
  JOIN (
      SELECT county, sum(cases) as cases FROM ny_state_window_m1 GROUP BY county ORDER BY cases DESC LIMIT 10
    ) x ON x.county = f.county

date,county,cases_per_100Kpop
2020-03-18,Nassau,13.48638538341
2020-03-16,Suffolk,4.26655542018
2020-03-11,Saratoga,0.87008348451
2020-03-15,Dutchess,2.37918822098
2020-03-15,Orange,1.55868447031
2020-03-19,New York City,151.53133837537
2020-03-07,Saratoga,0.87008348451
2020-03-17,Suffolk,5.68874056025
2020-03-18,Westchester,55.50353176104
2020-03-08,Rockland,0.61389426899


## Visualize Cases by State Choropleth Maps
* Join the data with `map_fips_dedup` to obtain the county centroid lat, long_

In [23]:
# Extract Day Number and county centroid lat, long_
wa_daynum = spark.sql("""select f.fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from wa_state_window f join map_fips_dedup m on m.fips = f.fips""")
wa_daynum.createOrReplaceTempView("wa_daynum")
ny_daynum = spark.sql("""select cast(f.fips as int) as fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from ny_state_window f join map_fips_dedup m on m.fips = f.fips""")
ny_daynum.createOrReplaceTempView("ny_daynum")
ny_daynum_m1 = spark.sql("""select cast(f.fips as int) as fips, f.county, f.date, f.day_num, cases as confirmed, cast(f.cases_per_100Kpop as int) as confirmed_per100K, deaths, cast(f.deaths_per_100Kpop as int) as deaths_per100K, m.lat, m.long_ from ny_state_window_m1 f join map_fips_dedup m on m.fips = f.fips""")
ny_daynum_m1.createOrReplaceTempView("ny_daynum_m1")

In [24]:
# Obtain Topography
topo_usa = 'https://vega.github.io/vega-datasets/data/us-10m.json'
topo_wa = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/WA-53-washington-counties.json'
topo_ny = 'https://raw.githubusercontent.com/deldersveld/topojson/master/countries/us-states/NY-36-new-york-counties.json'
us_counties = alt.topo_feature(topo_usa, 'counties')
wa_counties = alt.topo_feature(topo_wa, 'cb_2015_washington_county_20m')
ny_counties = alt.topo_feature(topo_ny, 'cb_2015_new_york_county_20m')

In [25]:
# Review WA
confirmed_wa = wa_daynum.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_wa['date'] = confirmed_wa['date'].astype(str)
deaths_wa = wa_daynum.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_wa['date'] = deaths_wa['date'].astype(str)

# Review NY
confirmed_ny = ny_daynum.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_ny['date'] = confirmed_ny['date'].astype(str)
deaths_ny = ny_daynum.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_ny['date'] = deaths_ny['date'].astype(str)

# Review NY -1 week
confirmed_ny_m1 = ny_daynum_m1.select("fips", "day_num", "date", "confirmed", "confirmed_per100K", "county").where("confirmed > 0").toPandas()
confirmed_ny_m1['date'] = confirmed_ny_m1['date'].astype(str)
deaths_ny_m1 = ny_daynum_m1.select("lat", "long_", "day_num", "date", "deaths", "deaths_per100K", "county").where("deaths > 0").toPandas()
deaths_ny_m1['date'] = deaths_ny_m1['date'].astype(str)

In [26]:
# State Choropleth Map Visualization Function
def map_state(curr_day_num, state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max):
  # Get date_str
  date_str = confirmed[confirmed['day_num'] == 101]['date'].head(1).item()
  
  # State
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # counties
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
  ).transform_lookup(
    lookup='id',
    from_=alt.LookupData(confirmed[(confirmed['confirmed_per100K'] > 0) & (confirmed['day_num'] == curr_day_num)], 'fips', ['confirmed_per100K', 'confirmed', 'county', 'date', 'fips'])  
  ).encode(
     color=alt.Color('confirmed_per100K:Q', scale=alt.Scale(type='log', domain=[confirmed_min, confirmed_max]), title='Confirmed per 100K'),
    tooltip=[
      alt.Tooltip('fips:O'),
      alt.Tooltip('confirmed:Q'),
      alt.Tooltip('confirmed_per100K:Q'),
      alt.Tooltip('county:N'),
      alt.Tooltip('date:N'),
    ],
  )

  # deaths by long, latitude
  points = alt.Chart(deaths[(deaths['deaths_per100K'] > 0) & (deaths['day_num'] == curr_day_num)]).mark_point(opacity=0.75, filled=True).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('sum(deaths_per100K):Q', scale=alt.Scale(type='symlog', domain=[deaths_min, deaths_max]), title='Deaths per 100K'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
    tooltip=[
      alt.Tooltip('lat'),
      alt.Tooltip('long_'),
      alt.Tooltip('deaths'),
      alt.Tooltip('county:N'),      
      alt.Tooltip('date:N'),      
    ],
  ).properties(
    # update figure title
    title=f'COVID-19 {state_txt} Confirmed Cases and Deaths per 100K by County [{curr_day_num}, {date_str}]'
  )

  return (base_state + base_state_counties + points)


| Factors | WA | NY | 
| ------- | -- | -- | 
| Educational Facilities Closed | 3/13/2020 | 3/18/2020 |
| Day 00 | 3/6/2020 | 3/11/2020 |
| Day 14 | 3/20/2020 | 3/25/2020 | 
| Max Cases | 794 | 20011 |
| Max Deaths | 68 | 280 |
| Max Cases per 100K | 50.55 | 1222.97 | 
| Max Deaths per 100K | 3.27 | 17.11 |

### WA State (cases and deaths per 100K)

In [29]:
map_state(101, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

In [30]:
map_state(107, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

In [31]:
map_state(114, 'WA', wa_counties, confirmed_wa, 1, 60, deaths_wa, 1, 5)

### NY State (cases and deaths per 100K)

In [33]:
map_state(101, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

In [34]:
map_state(107, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

In [35]:
map_state(114, 'NY', ny_counties, confirmed_ny, 1, 1500, deaths_ny, 1, 20)

## COVID-19 Confirmed Cases and Deaths by WA and NY County Slider

In [37]:
# State Choropleth Map Visualization Function
def map_state_slider(state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max, state_fips):
  # Pivot confirmed data by day_num
  confirmed_pv = confirmed[['fips', 'day_num', 'confirmed']].copy()
  confirmed_pv['fips'] = confirmed_pv['fips'].astype(str)
  confirmed_pv['day_num'] = confirmed_pv['day_num'].astype(str)
  confirmed_pv['confirmed'] = confirmed_pv['confirmed'].astype('int64')
  confirmed_pv = confirmed_pv.pivot_table(index='fips', columns='day_num', values='confirmed', fill_value=0).reset_index()

  # Pivot deaths data by day_num
  deaths_pv = deaths[['lat', 'long_', 'day_num', 'deaths']].copy()
  deaths_pv['day_num'] = deaths_pv['day_num'].astype(str)
  deaths_pv['deaths'] = deaths_pv['deaths'].astype('int64')
  deaths_pv = deaths_pv.pivot_table(index=['lat', 'long_'], columns='day_num', values='deaths', fill_value=0).reset_index()

  # Extract column names for slider
  column_names = confirmed_pv.columns.tolist()

  # Remove first element (`fips`)
  column_names.pop(0)

  # Convert to int
  column_values = [None] * len(column_names)
  for i in range(0, len(column_names)): column_values[i] = int(column_names[i]) 
  
  # Disable max_rows to see more data
  alt.data_transformers.disable_max_rows()

  # Topographic information
  us_states = alt.topo_feature(topo_usa, 'states')
  us_counties = alt.topo_feature(topo_usa, 'counties')

  # state county boundaries
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # Slider choices
  min_day_num = column_values[0]
  max_day_num = column_values[len(column_values)-1]
  slider = alt.binding_range(min=min_day_num, max=max_day_num, step=1)
  slider_selection = alt.selection_single(fields=['day_num'], bind=slider, name="day_num", init={'day_num':min_day_num})


  # Confirmed cases by county
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
      stroke='black',
      strokeWidth=0.05
  ).transform_lookup(
      lookup='id',
      from_=alt.LookupData(confirmed_pv, 'fips', column_names)  
  ).transform_fold(
      column_names, as_=['day_num', 'confirmed']
  ).transform_calculate(
      state_id = "(datum.id / 1000)|0",
      day_num = 'parseInt(datum.day_num)',
      confirmed = 'isValid(datum.confirmed) ? datum.confirmed : -1'
  ).encode(
      color = alt.condition(
          'datum.confirmed > 0',      
          alt.Color('confirmed:Q', scale=alt.Scale(domain=(confirmed_min, confirmed_max), type='symlog')),
          alt.value('white')
        )  
  ).properties(
    # update figure title
    title=f'COVID-19 WA State Confirmed Cases by County'
  ).transform_filter(
      (alt.datum.state_id)==state_fips
  ).transform_filter(
      slider_selection
  )

  # deaths by long, latitude
  points = alt.Chart(
    deaths_pv
  ).mark_point(
    opacity=0.75, filled=True
  ).transform_fold(
    column_names, as_=['day_num', 'deaths']
  ).transform_calculate(
      day_num = 'parseInt(datum.day_num)',
      deaths = 'isValid(datum.deaths) ? datum.deaths : -1'  
  ).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('deaths:Q', scale=alt.Scale(domain=(deaths_min, deaths_max), type='symlog'), title='deaths'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
  ).add_selection(
      slider_selection
  ).transform_filter(
      slider_selection
  )

  # confirmed cases (base_counties) and deaths (points)
  return (base_state + base_state_counties + points) 

| Factors | WA | NY | 
| ------- | -- | -- | 
| Educational Facilities Closed | 3/13/2020 | 3/18/2020 |
| Day 00 | 3/6/2020 | 3/11/2020 |
| Day 14 | 3/20/2020 | 3/25/2020 | 
| Max Cases | 794 | 20011 |
| Max Deaths | 68 | 280 |
| Max Cases per 100K | 50.55 | 1222.97 | 
| Max Deaths per 100K | 3.27 | 17.11 |

In [39]:
map_state_slider('WA', wa_counties, confirmed_wa, 1, 800, deaths_wa, 1, 70, 53)

In [40]:
map_state_slider('NY', ny_counties, confirmed_ny, 1, 21000, deaths_ny, 1, 300, 36)

## COVID-19 Confirmed Cases and Deaths by WA and NY County Map and Graph

In [42]:
# map_state_graph
def map_state_graph(state_txt, state_counties, confirmed, confirmed_min, confirmed_max, deaths, deaths_min, deaths_max, state_fips):
  
  # pivot confirmed cases (by date)
  confirmed_pv2 = confirmed[['fips', 'date', 'confirmed']].copy()
  confirmed_pv2['fips'] = confirmed_pv2['fips'].astype(str)
  confirmed_pv2['date'] = confirmed_pv2['date'].astype(str)
  confirmed_pv2['confirmed'] = confirmed_pv2['confirmed'].astype('int64')
  confirmed_pv2 = confirmed_pv2.pivot_table(index='fips', columns='date', values='confirmed', fill_value=0).reset_index()

  # pivot deaths
  deaths_pv2 = deaths[['lat', 'long_', 'date', 'deaths']].copy()
  deaths_pv2['date'] = deaths_pv2['date'].astype(str)
  deaths_pv2['deaths'] = deaths_pv2['deaths'].astype('int64')
  deaths_pv2 = deaths_pv2.pivot_table(index=['lat', 'long_'], columns='date', values='deaths', fill_value=0).reset_index()

  # Extract column names for slider
  column_names2 = confirmed_pv2.columns.tolist()

  # Remove first element (`fips`)
  column_names2.pop(0)

  # date selection
  pts = alt.selection(type="single", encodings=['x'])

  # State
  base_state = alt.Chart(state_counties).mark_geoshape(
      fill='white',
      stroke='lightgray',
  ).properties(
      width=800,
      height=600,
  ).project(
      type='mercator'
  )

  # State Counties
  base_state_counties = alt.Chart(us_counties).mark_geoshape(
    stroke='black',
    strokeWidth=0.05,
  ).transform_lookup(
    lookup='id',
   from_=alt.LookupData(confirmed_pv2, 'fips', column_names2)
   ).transform_fold(
     column_names2, as_=['date', 'confirmed']
  ).transform_calculate(
      state_id = "(datum.id / 1000)|0",
      date = 'datum.date',
      confirmed = 'isValid(datum.confirmed) ? datum.confirmed : -1'
  ).encode(
       color = alt.condition(
          'datum.confirmed > 0',      
          alt.Color('confirmed:Q', scale=alt.Scale(domain=(confirmed_min, confirmed_max), type='symlog')),
          alt.value('white')
        )  
  ).transform_filter(
    pts
  ).transform_filter(
      (alt.datum.state_id)==state_fips
  )

  # Bar Graph
  bar = alt.Chart(confirmed).mark_bar().encode(
      x='date:N',
      y='confirmed_per100K:Q',
      color=alt.condition(pts, alt.ColorValue("steelblue"), alt.ColorValue("grey"))
  ).properties(
      width=800,
      height=200,
      title='Confirmed Cases per 100K'
  ).add_selection(pts)

  # Deaths
  points = alt.Chart(deaths).mark_point(opacity=0.75, filled=True).encode(
    longitude='long_:Q',
    latitude='lat:Q',
    size=alt.Size('sum(deaths):Q', scale=alt.Scale(domain=[deaths_min, deaths_max]), title='Deaths'),
    color=alt.value('#BD595D'),
    stroke=alt.value('brown'),
    tooltip=[
      alt.Tooltip('lat'),
      alt.Tooltip('long_'),
      alt.Tooltip('deaths'),
      alt.Tooltip('county:N'),      
      alt.Tooltip('date:N'),      
    ],
  ).properties(
    # update figure title
    title=f'COVID-19 Confirmed Cases and Deaths by County'
  ).transform_filter(
      pts
  )

  return (base_state + base_state_counties + points) & bar

In [43]:
map_state_graph('WA', wa_counties, confirmed_wa, 1, 800, deaths_wa, 1, 70, 53)

In [44]:
map_state_graph('NY', ny_counties, confirmed_ny, 1, 21000, deaths_ny, 1, 300, 36)

In [45]:
map_state_graph('NY', ny_counties, confirmed_ny_m1, 1, 4500, deaths_ny, 1, 70, 36)